# Lab 03: Cây quyết định

- Họ tên:
- MSSV:

<font color='red'>**LƯU Ý:**</font>

- Bài làm của sinh viên phải do chính bản thân sinh viên tự làm, có thể trao đổi và tham khảo ý tưởng nhưng không được sao chép (một phần hoặc toàn bộ) code hoặc lời giải từ bất cứ người nào khác. Nếu vi phạm sẽ bị <font color='red'>0đ</font> bài tập này.
- Các bạn có thể tạo thêm các cell trong quá trình code, tuy nhiên các bạn vui lòng <font color='red'>không xóa các cell code mặc định và các cell test case</font> (vì có thể ảnh hưởng đến kết quả khi chấm bài).
- Các test case được đưa ra chỉ nhằm mục đích giúp các bạn test code của mình, <font color='red'>**việc pass các test case này không đồng nghĩa với việc lời giải của các bạn sẽ đạt điểm tối đa**</font>. 

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; trong file, từ `TODO` để cho biết những phần mà bạn cần phải làm.

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của các bạn; do đó, trước khi nộp bài, các bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, các bạn tạo thư mục nộp bài theo cấu trúc sau:

- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - File `HW03.ipynb` (Đổi tên file notebook này theo MSSV của bạn)

Cuối cùng, các bạn nén thư mục `MSSV` này lại và nộp ở link trên moodle. **Đuôi của file nén phải là .zip (chứ không được .rar hay gì khác).**

<font color=red>Các bạn lưu ý tuân thủ chính xác qui định nộp bài ở trên.</font>

**Nội dung bài tập**

Bài tập 3 là bài tập cá nhân. Trong bài này, bạn sẽ cài đặt thuật toán học máy: 
1. Cây quyết định (Decision tree)

### Tải những thư viện cần thiết

In [27]:

import os
from zlib import adler32
from typing import Tuple, List
import requests
import shutil
import random

import scipy
import numpy as np
import sklearn.datasets as datasets
import matplotlib.pyplot as plt

In [28]:
# Init seed
seed = 2024
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

### Tải tập dữ liệu về thuốc

Hãy tưởng tượng rằng bạn là một nhà nghiên cứu y khoa đang biên soạn dữ liệu cho một nghiên cứu. Bạn đã thu thập dữ liệu về một nhóm bệnh nhân, tất cả đều mắc cùng một căn bệnh. Trong quá trình điều trị, mỗi bệnh nhân đều đáp ứng với một trong 5 loại thuốc, Thuốc A, Thuốc B, Thuốc c, Thuốc x và thuốc y.

Một phần công việc của bạn là xây dựng một mô hình để tìm ra loại thuốc nào có thể phù hợp với bệnh nhân tương lai mắc cùng một căn bệnh. Các đặc điểm của tập dữ liệu này là Tuổi, Giới tính, Huyết áp và Cholesterol của bệnh nhân, và mục tiêu là loại thuốc mà mỗi bệnh nhân đáp ứng.

Đây là một mẫu của bộ phân loại đa lớp và bạn có thể sử dụng tập dữ liệu để xây dựng một cây quyết định, sau đó sử dụng nó để dự đoán lớp của một bệnh nhân chưa biết hoặc kê đơn thuốc cho một bệnh nhân mới.

### Đọc tập dữ liệu về thuốc

Viết hàm `drug_dataloader` để load tập dữ liệu từ tập tin `drug200_numeric.csv` và trả về hai mảng numpy array `features` và `classes` đại diện cho đặc trưng và lớp tương ứng. Lưu ý, không sử Pandas. 

In [29]:
def drug_dataloader(file_path="drug200_numeric.csv"):
    # YOUR CODE HERE
    features = []
    classes = []
    with open(file_path, "r") as file:
        # Skip the header line
        next(file)
        for line in file.readlines():
            data = line.strip().split(',')
            features.append([float(value) for value in data[:-1]])
            classes.append(int(data[-1]))
    features = np.array(features)
    classes = np.array(classes)
    
    #raise NotImplementedError()
    return features, classes

features, classes = drug_dataloader()

In [30]:
print(adler32(str(features[0][:5] + features[1][:5] + features[2][:5]).encode()))

2319320757


In [31]:
# TEST
assert (
    adler32(str(features[0][:5] + features[1][:5] + features[2][:5]).encode())
    == 1140262165
)

AssertionError: 

### Phân chia tập train-valid-test

In [32]:
def train_test_split(
    X: np.ndarray, y: np.ndarray, test_ratio=0.33
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Function for proceduring train, and test sets.

    Args:
        X (np.ndarray): input features.
        y (np.ndarray): input labels.
        test_ratio (float, optional): ratio size for test sets. Defaults to 0.33.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]: features, label for train, test respectively. 
        They are represented as numpy array.
    """
    # YOUR CODE HERE
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    test_size = int(len(indices) * test_ratio)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]

    X_train = X[train_indices]
    X_test = X[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]


    
    #raise NotImplementedError()
    return (
        X_train,
        X_test,
        y_train,
        y_test,
    )


# Phân chia tập train, test.
X_train, X_test, y_train, y_test = train_test_split(features, classes)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((134, 5), (66, 5), (134,), (66,))

In [33]:
# TEST
SP = X_train.shape + X_test.shape + y_train.shape + y_test.shape
assert adler32(str(SP).encode()) == 889979968

## Cây quyết định

### Độ lợi thông tin

Thông tin kỳ vọng (entropy):

$$Entropy=-\sum_{i}^{n}p_ilog_{2}(p_i)$$

Hàm entropy đạt giá trị nhỏ nhất nếu có một giá trị $p_i=1$, đạt giá trị lớn nhất nếu tất cả các $p_i$ bằng nhau. Những tính chất này của hàm entropy khiến nó được sử dụng trong việc đo độ hỗn loạn của một phép phân chia của ID3.

In [34]:
def entropy(counts: List, n_samples: int) -> float:
    """Function for calculating entropy

    Args:
        counts (list): list number of samples in each class.
        n_samples (int): number of data samples.

    Return:
        entropy (float).
    """
    # YOUR CODE HERE
    probabilities = [count / n_samples for count in counts if count > 0]
    return -sum(p * np.log2(p) for p in probabilities)
    # raise NotImplementedError()
    return entropy

In [35]:
def entropy_of_one_division(division: np.ndarray) -> Tuple[float, int]:
    """Function for calculating entropy of a divided group of data.

    Please note that data may have multiple classes.

    Args:
        division (np.ndarray): input divided group of data.

    Returns:
        Tuple[float, int]: entropy of a divided group of data.
    """
    n_samples = len(division)
    n_classes = set(division)

    counts = []
    # count samples in each class then store it to list counts
    # YOUR CODE HERE
    counts = [np.sum(division == cls) for cls in n_classes]
    #raise NotImplementedError()
    return entropy(counts, n_samples), n_samples

In [36]:
def get_entropy(y_predict: np.ndarray, y: np.ndarray):
    """Get entropy for a split.

    Args:
        y_predict (np.ndarray): the split decision by cutoff, True/Fasle.
        y (np.ndarray): grouth truth.

    Returns:
        s (float): entropy of input split, as real-number represented with float type.
    """
    n = len(y)
    entropy_true, n_true = entropy_of_one_division(
        y[y_predict]
    )  # left hand side entropy
    
    entropy_false, n_false = entropy_of_one_division(
        y[~y_predict]
    )  # right hand side entropy
    # overall entropy
    
    # s=?
    # YOUR CODE HERE
    s = (n_true / n) * entropy_true + (n_false / n) * entropy_false
    # raise NotImplementedError()

Độ lợi thông tin phân lớp tập D theo thuộc tính A:
$$ Gain(A)=Entropy(D)-Entropy_{A}(D)$$

Trong ID3, tại mỗi node, thuộc tính được chọn được xác định dựa trên là thuộc tính khiến cho information gain đạt giá trị lớn nhất.

Các thuộc tính của tập Iris đều có giá trị liên tục. Do đó ta cần rời rạc hóa cho từng thuộc tính. Cách đơn giản là sử dụng một ngưỡng `cutoff` chia giá trị của dữ liệu trên mỗi thuộc tính sẽ làm 2 phần: `<cutoff` và `>=cutoff`.

Để tìm ngưỡng `cutoff` tốt nhất cho mỗi thuộc tính ta lần lượt thay `cutoff` bằng các giá trị của thuộc tính sau đó tính entropy, `cutoff` tốt nhất khi entropy bé nhất 

$$\left(\arg\min Entropy_{A}(D)\right)$$

### Cài đặt

#### Tìm một phân hoạch với một độ lợi thông tin cho trước

In [46]:
def find_best_split(
    col_data: np.ndarray, y: np.ndarray
) -> Tuple[np.float64, np.float64]:
    """Function for calculating minimum entropy for a given attributes and its label.

    Args:
        col_data (np.ndarray): input column data in training dataset.
        y (np.ndarray): given label in the training dataset.

    Returns:
        min_entropy, cutoff (Tuple[np.float64, np.float64]): the minimum entropy, and cut-off value.
    """
    min_entropy = float("inf")
    cutoff = None

    # Loop through col_data find cutoff where entropy is minimum
    for value in set(col_data):
        y_predict = col_data < value
        my_entropy = get_entropy(y_predict, y)

        # min entropy=?, cutoff=?
        # YOUR CODE HERE
        if my_entropy < min_entropy:
            min_entropy = my_entropy
            cutoff = value

        # raise NotImplementedError()

    # Return min entropy, and cutoff
    return min_entropy, cutoff

In [43]:
def find_best_split_of_all(
    X: np.ndarray, y: np.ndarray
) -> Tuple[np.float64, np.float64, np.float64]:
    """Function for finding one split given an information gain.

    Args:
        X (np.ndarray): input training dataset.
        y (np.ndarray): given label in the training dataset.

    Returns:
        col_idx, cutoff, min_entropy (Tuple[np.float64, np.float64, np.float64]): return the index 
        of column with minimum entropy and cut-off value.
    """
    col_idx = None
    min_entropy = float("inf")
    cutoff = None

    for idx, col_data in enumerate(X.T):
        entropy, cur_cutoff = find_best_split(col_data, y)

        if entropy == 0:  # best entropy
            return idx, cur_cutoff, entropy
        elif entropy <= min_entropy:
            min_entropy = entropy
            col_idx = idx
            cutoff = cur_cutoff

    return col_idx, cutoff, min_entropy

#### Khớp dữ liệu

In [44]:
def dtfit(X: np.ndarray, y: np.ndarray, depth: int = 0) -> dict:
    """Function for data-fitting with Decision Tree

    Node: each node represented by cutoff value and column index, value and children.
         - cutoff value is thresold where you divide your attribute,
         - column index is your data attribute index,
         - value of node is mean value of label indexes,
           if a node is leaf all data samples will have same label.

    Note that: we divide each attribute into 2 part => each node will have 2 children: left, right.

    Args:
        X (np.ndarray): training data
        y (np.ndarray): label of training data
        depth (int, optional): depth of decision tree after training. Defaults to 0.

    Returns:
        node (dict): return the node that contains cutoff value and column index, value and children.
    """
    # Stop conditions: if all value of y are the same
    if np.all(y == y[0]):
        return {"val": y[0]}

    # find one split given an information gain
    col_idx, cutoff, entropy = find_best_split_of_all(X, y)

    y_left = y[X[:, col_idx] < cutoff]
    y_right = y[X[:, col_idx] >= cutoff]
    X_left = X[X[:, col_idx] < cutoff]
    X_right = X[X[:, col_idx] >= cutoff]

    node = {
        "index_col": col_idx,
        "cutoff": cutoff,
        "val": np.mean(y),
        "left": None,
        "right": None,
    }

    node["left"] = dtfit(X_left, y_left, depth + 1)
    node["right"] = dtfit(X_right, y_right, depth + 1)

    return node

In [45]:
# Khớp dữ liệu
tree = dtfit(X_train, y_train)

TypeError: '<' not supported between instances of 'NoneType' and 'float'

#### Dự đoán

In [47]:
def _dtpredict(tree: dict, row: np.ndarray) -> int:
    """Function for making prediction use trained DT tree on given observation.

    Args:
        tree (dict): trained DT model which presented as a dict.
        row (np.ndarray): given observation which presented as numpy's array.

    Returns:
        val (int): return the value of node which is mean value of label indexes.
    """
    cur_layer = tree
    while "cutoff" in cur_layer:
        if row[cur_layer["index_col"]] < cur_layer["cutoff"]:
            cur_layer = cur_layer["left"]
        else:
            cur_layer = cur_layer["right"]
    return cur_layer.get("val", False)

In [48]:
# TEST
given_observation = features[81]
groud_truth = classes[81]
assert groud_truth == _dtpredict(tree, given_observation)

NameError: name 'tree' is not defined

In [ ]:
def dtpredict(tree: dict, data: np.ndarray) -> List:
    """Function for making prediction with trained DT model on given input observations.

    Args:
        tree (dict): trained DT model which presented as a dict.
        data (np.ndarray): input input observations.

    Returns:
        pred (List): list of predicted label for input observations.
    """
    pred = []
    for _, col in enumerate(data):
        pred.append(_dtpredict(tree, col))
    return pred

In [ ]:
# TEST
given_observations = features[81:118]
groud_truth = classes[81:118]
res = (groud_truth == dtpredict(tree, given_observations))
assert adler32(str(res).encode()) == 794314306

#### Đánh giá mô hình

In [ ]:
def tpfptnfn_cal(
    y_test: np.ndarray, y_pred: np.ndarray, positive_class: int = 1
) -> Tuple[int, int, int, int]:
    """Function for calculating elements of confusion matrix.

    Args:
        y_test (np.ndarray): groud truth.
        y_pred (np.ndarray): predicted label.
        positive_class (int, optional): wanted calculating class. Defaults to 1.

    Returns:
        true_positives, false_positives, true_negatives, false_negatives (Tuple[int, int, int, int]): Four 
        basic number for constructing confusion matrix  including true positives, false positives, true negatives, 
        and false negatives.
    """
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    # Calculate true positives, false positives, false negatives, and true negatives
    # YOUR CODE HERE
    raise NotImplementedError()
    return true_positives, false_positives, true_negatives, false_negatives

In [ ]:
# TEST
tp, fp, tn, fn = tpfptnfn_cal([1, 0, 1, 1, 0, 0, 1], [1, 0, 0, 1, 0, 1, 1])
assert adler32((str(tp) + str(fp) + str(tn) + str(fn)).encode()) == 33030344

In [ ]:
# Dự đoán
pred = dtpredict(tree, X_test)

In [ ]:
def calculate_metrics(
    y_test: np.ndarray, y_pred: np.ndarray
) -> Tuple[float, float, float, float]:
    """Function for calculating metrics of classification problem including accuracy, recall, 
    precision, and f1-score.

    Args:
        y_test (np.ndarray): groud truth.
        y_pred (np.ndarray): predicted label.

    Returns:
        acc, precision, recall, f1 (Tuple[float, float, float, float]): return four values 
        for each metric: accuracy, recall, precision, and f1-score.
    """
    acc = 0
    precision = 0
    recall = 0
    f1 = 0

    noc = len(np.unique(y_test))  # number of classes

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Nghiên cứu về ý nghĩa các độ đo

Bạn hãy trình ý nghĩa từng độ đo, bao gồm accuracy, precision, recall và f-score.

YOUR ANSWER HERE